In [1]:
from cvxpylayers.torch import CvxpyLayer
import numpy as np
import cvxpy as cp
import torch
import torch.nn as nn
import match_net_torch as mn

In [5]:
true_bids = torch.tensor([[[ 6.,  0.,  0.,  7., 14., 13.,  0.],
         [ 0., 11.,  6.,  0.,  0.,  0., 13.]],

        [[16.,  0.,  0., 11.,  8.,  5.,  0.],
         [ 0.,  9.,  7.,  0.,  0.,  0., 14.]],

        [[ 8.,  0.,  0.,  9., 15.,  8.,  0.],
         [ 0.,  8.,  9.,  0.,  0.,  0., 13.]],

        [[ 7.,  0.,  0.,  9., 15.,  9.,  0.],
         [ 0., 10., 10.,  0.,  0.,  0., 10.]],

        [[13.,  0.,  0.,  8., 12.,  7.,  0.],
         [ 0., 17.,  5.,  0.,  0.,  0.,  8.]],

        [[10.,  0.,  0., 12., 10.,  8.,  0.],
         [ 0.,  7., 12.,  0.,  0.,  0., 11.]],

        [[10.,  0.,  0.,  7.,  9., 14.,  0.],
         [ 0.,  8.,  8.,  0.,  0.,  0., 14.]],

        [[15.,  0.,  0.,  8.,  7., 10.,  0.],
         [ 0.,  9., 10.,  0.,  0.,  0., 11.]],

        [[ 9.,  0.,  0., 15., 10.,  6.,  0.],
         [ 0.,  7.,  9.,  0.,  0.,  0., 14.]],

        [[ 8.,  0.,  0., 11.,  8., 13.,  0.],
         [ 0., 10., 10.,  0.,  0.,  0., 10.]],

        [[13.,  0.,  0., 12.,  8.,  7.,  0.],
         [ 0.,  8.,  7.,  0.,  0.,  0., 15.]],

        [[ 9.,  0.,  0.,  9.,  6., 16.,  0.],
         [ 0.,  9.,  9.,  0.,  0.,  0., 12.]],

        [[ 7.,  0.,  0., 15., 11.,  7.,  0.],
         [ 0.,  9., 12.,  0.,  0.,  0.,  9.]],

        [[11.,  0.,  0.,  8., 15.,  6.,  0.],
         [ 0., 10.,  7.,  0.,  0.,  0., 13.]],

        [[10.,  0.,  0.,  8.,  8., 14.,  0.],
         [ 0., 16.,  4.,  0.,  0.,  0., 10.]],

        [[10.,  0.,  0., 11., 11.,  8.,  0.],
         [ 0., 13.,  7.,  0.,  0.,  0., 10.]],

        [[ 9.,  0.,  0., 13., 10.,  8.,  0.],
         [ 0., 11.,  8.,  0.,  0.,  0., 11.]],

        [[16.,  0.,  0.,  9.,  9.,  6.,  0.],
         [ 0.,  8.,  9.,  0.,  0.,  0., 13.]],

        [[ 9.,  0.,  0., 11.,  9., 11.,  0.],
         [ 0.,  9.,  9.,  0.,  0.,  0., 12.]],

        [[11.,  0.,  0., 13.,  3., 13.,  0.],
         [ 0., 11., 13.,  0.,  0.,  0.,  6.]],

        [[11.,  0.,  0., 10.,  9., 10.,  0.],
         [ 0.,  8.,  6.,  0.,  0.,  0., 16.]],

        [[ 7.,  0.,  0., 15.,  8., 10.,  0.],
         [ 0., 15.,  9.,  0.,  0.,  0.,  6.]],

        [[ 8.,  0.,  0., 13.,  7., 12.,  0.],
         [ 0., 12.,  8.,  0.,  0.,  0., 10.]],

        [[11.,  0.,  0., 11.,  8., 10.,  0.],
         [ 0.,  8., 10.,  0.,  0.,  0., 12.]],

        [[10.,  0.,  0.,  8., 12., 10.,  0.],
         [ 0., 13., 12.,  0.,  0.,  0.,  5.]],

        [[12.,  0.,  0., 10.,  9.,  9.,  0.],
         [ 0., 11.,  6.,  0.,  0.,  0., 13.]],

        [[12.,  0.,  0., 10., 10.,  8.,  0.],
         [ 0.,  6., 12.,  0.,  0.,  0., 12.]],

        [[ 9.,  0.,  0., 10.,  7., 14.,  0.],
         [ 0., 11., 10.,  0.,  0.,  0.,  9.]],

        [[10.,  0.,  0.,  9., 11., 10.,  0.],
         [ 0.,  9., 11.,  0.,  0.,  0., 10.]],

        [[ 6.,  0.,  0., 14.,  9., 11.,  0.],
         [ 0.,  9.,  8.,  0.,  0.,  0., 13.]],

        [[14.,  0.,  0.,  6.,  7., 13.,  0.],
         [ 0.,  6., 16.,  0.,  0.,  0.,  8.]],

        [[ 4.,  0.,  0.,  9., 11., 16.,  0.],
         [ 0., 13.,  9.,  0.,  0.,  0.,  8.]]])

In [6]:
central_alloc = torch.tensor([[[ 3.8146e+00,  2.8928e-06,  2.9627e-07,  3.7417e+00,  3.7906e+00,
           6.6339e+00,  6.4839e-06, -6.2760e-06,  7.5107e+00,  6.0000e+00,
          -3.5168e-06,  1.1025e-05, -2.7067e-06,  4.2811e+00],
         [ 3.2110e+00, -3.6610e-07, -2.5236e-06,  4.6912e+00,  2.9122e+00,
           2.5693e+00, -1.3029e-06, -1.5156e-06,  7.9032e+00,  7.0000e+00,
           3.3019e-06, -1.6064e-06,  2.1305e-06,  2.0405e+00],
         [ 2.3127e+00, -1.4067e-06, -1.3643e-07,  2.1914e+00,  1.3564e+00,
           1.1123e-01, -1.6188e-06,  1.1300e-06,  8.0000e+00,  6.5779e+00,
          -2.6954e-06,  1.5160e-07, -1.1056e-06,  5.5678e-02],
         [ 2.8803e+00,  1.8331e-06, -7.9659e-07,  7.9701e+00,  5.0430e+00,
           1.4516e-06, -4.6463e-07,  8.6972e-07,  9.9532e+00,  1.0000e+01,
          -4.5381e-07, -7.0034e-07,  2.9506e-06,  1.3673e-06],
         [ 7.7426e+00, -2.6261e-06, -1.7734e-06,  5.8210e-01,  6.1914e+00,
           6.0233e+00, -3.4839e-06, -4.7134e-07,  1.2743e+01,  5.0000e+00,
           7.8306e-06, -1.4620e-06,  3.1200e-06,  4.1405e-01],
         [ 2.6100e+00, -9.3522e-07, -2.5264e-06,  7.7965e+00,  3.5193e+00,
           5.4365e+00, -7.4609e-07, -6.4056e-07,  7.0000e+00,  1.1802e+01,
           8.6819e-07, -6.1979e-07,  4.3352e-06,  2.3019e+00],
         [ 3.2424e+00, -1.2774e-07, -1.7929e-07,  5.5112e+00,  3.1608e+00,
           6.1467e+00, -6.8560e-07,  3.1848e-07,  6.7701e+00,  8.0000e+00,
           1.6713e-08, -4.6287e-07, -9.6180e-07,  4.0249e+00],
         [ 3.3592e+00,  7.3047e-07, -1.8393e-06,  7.1640e+00,  1.7769e+00,
           3.7438e+00, -8.2089e-08, -2.0888e-06,  7.1175e+00,  1.0000e+01,
           3.3377e-06, -5.1623e-07, -3.3929e-06,  2.8893e+00],
         [ 1.2904e-05, -4.7865e-06,  6.7737e-06,  1.5532e+00, -2.6049e-06,
           3.6437e+00, -1.9358e-06,  8.7834e-06,  7.0000e+00,  8.5531e+00,
          -4.6054e-06, -3.0057e-06, -9.2987e-07,  3.6437e+00],
         [ 3.3928e+00, -1.7548e-06, -4.4187e-07,  6.9935e+00,  8.0000e+00,
           9.9052e+00,  4.1331e-07, -5.8051e-07,  7.4897e+00,  1.0000e+01,
           5.0947e-07, -6.1965e-07, -1.3881e-06,  2.9957e+00],
         [ 3.0816e+00, -4.5938e-06, -2.1571e-06,  4.2678e+00,  3.2503e+00,
           4.1437e+00,  2.2456e-06,  3.5692e-07,  7.3644e+00,  7.0000e+00,
          -2.4988e-07, -4.1618e-06,  5.6924e-06,  2.4439e+00],
         [ 3.1202e+00, -4.9925e-07,  3.1222e-07,  4.9447e+00,  6.0000e+00,
           9.1846e+00,  1.7661e-06,  1.4115e-06,  8.0698e+00,  9.0000e+00,
          -1.2034e-06, -8.6897e-07, -3.9829e-06,  4.0789e+00],
         [ 3.0391e+00, -3.8409e-06,  1.7588e-06,  8.2434e+00,  3.5667e+00,
           4.1671e+00,  3.5370e-06, -2.4189e-06,  8.1855e+00,  1.2000e+01,
          -2.3916e-08,  3.4804e-06, -6.8661e-07,  1.9902e+00],
         [ 2.8810e+00, -2.8993e-07, -6.7593e-07, -6.0705e-07,  5.1894e+00,
           6.0000e+00, -3.2332e-07,  4.6461e-07,  9.8810e+00,  7.0000e+00,
          -4.2558e-08,  1.7912e-07,  3.3563e-07,  8.1064e-01],
         [ 7.3029e+00, -3.8352e-06, -3.7477e-07,  2.3964e+00,  1.1370e+00,
           4.6789e+00,  5.8716e-06,  7.4848e-07,  9.0692e+00,  4.0000e+00,
           6.5666e-06, -1.0967e-05,  7.0611e-06,  3.7047e+00],
         [ 5.4227e+00,  9.0482e-06, -4.9312e-06,  2.4052e+00,  2.9937e+00,
           4.9896e+00, -5.3627e-06, -1.4803e-06,  1.0323e+01,  7.0000e+00,
           2.0147e-06, -1.1753e-05,  1.1053e-05,  2.3018e+00],
         [ 1.6262e+00, -6.8111e-06,  1.8167e-05,  1.3218e+00,  2.1780e-06,
           5.9491e+00, -4.4327e-06, -4.8004e-07,  8.3045e+00,  8.0000e+00,
          -3.1964e-06,  1.6114e-05,  1.5563e-05,  5.9492e+00],
         [ 3.2390e+00,  1.0460e-06,  8.4847e-07,  8.4201e+00,  4.4902e+00,
           2.4155e+00, -7.2834e-07,  9.6638e-07,  7.1279e+00,  9.0000e+00,
          -2.0974e-07,  1.6088e-06, -1.1334e-06,  1.2342e+00],
         [ 2.9812e+00,  5.9832e-07,  1.4261e-06,  5.3010e+00,  2.9627e+00,
           5.1031e+00,  6.1608e-06, -3.0440e-07,  7.8417e+00,  9.0000e+00,
          -5.0905e-06,  4.1503e-06, -3.1199e-06,  3.3020e+00],
         [ 3.3063e+00,  2.7721e-06, -1.7592e-06,  7.9539e+00,  3.0000e+00,
           5.5591e+00, -2.4780e-06, -2.2689e-06,  9.7437e+00,  1.3000e+01,
          -4.1414e-06, -2.8763e-06, -1.6795e-06,  3.9503e+00],
         [ 2.8772e+00,  7.4644e-07,  2.7424e-07,  2.4471e+00,  3.7018e+00,
           5.7414e+00, -2.7966e-07,  3.7880e-07,  6.9014e+00,  6.0000e+00,
           1.9776e-07, -5.9973e-07,  6.8755e-07,  2.5110e+00],
         [ 5.2946e+00,  1.4426e-06, -3.0679e-07,  4.0490e+00,  8.0000e+00,
           7.9094e+00,  4.3738e-07, -8.0532e-07,  1.2539e+01,  9.0000e+00,
           4.5401e-07,  6.1904e-07,  3.5174e-07,  2.2029e+00],
         [ 5.0155e+00,  2.7547e-05,  7.9767e-06,  3.3355e+00,  1.5274e+00,
           4.3215e+00, -1.6691e-06, -2.2875e-06,  1.0432e+01,  8.0000e+00,
           9.9139e-06,  7.3740e-07,  5.6562e-06,  3.5461e+00],
         [ 3.1271e+00,  4.1173e-06, -1.3066e-06,  7.8454e+00,  2.6675e+00,
           4.3635e+00,  7.5655e-07, -2.6199e-06,  6.3644e+00,  1.0000e+01,
           4.3083e-06,  5.1275e-07,  3.6615e-06,  2.7787e+00],
         [ 3.2436e+00, -1.2321e-05, -4.4858e-06,  4.3573e+00,  3.4290e+00,
           5.1334e+00,  1.5383e-05,  5.5003e-07,  1.2169e+01,  1.2000e+01,
          -2.3826e-05,  4.5158e-06, -4.6803e-06,  2.9869e+00],
         [ 4.5115e+00,  7.4492e-07, -1.0362e-06,  2.6762e+00,  2.8347e+00,
           4.5085e+00, -1.0114e-06, -1.1358e-06,  8.5742e+00,  6.0000e+00,
          -1.3457e-07,  1.3008e-08,  4.8147e-07,  2.4126e+00],
         [ 2.6423e+00, -1.5928e-05,  7.2998e-06,  7.2135e+00,  6.6159e+00,
           7.6285e+00,  2.6729e-07,  1.0926e-05,  6.0000e+00,  1.0571e+01,
           4.4474e-07, -3.5348e-06,  3.0564e-06,  1.0127e+00],
         [ 3.3457e+00, -8.0668e-07, -5.8021e-07,  5.4630e+00,  7.0000e+00,
           9.7529e+00,  4.0477e-07,  9.0414e-07,  8.3418e+00,  1.0000e+01,
           2.7137e-07, -9.5540e-07, -1.4408e-06,  3.2120e+00],
         [ 3.2708e+00, -4.6815e-06, -1.0527e-06,  6.4445e+00,  4.7801e+00,
           6.5225e+00,  6.9667e-07,  6.4844e-07,  8.1640e+00,  1.1000e+01,
          -3.2969e-06,  2.5041e-07, -5.1173e-06,  2.0800e+00],
         [ 3.0347e+00,  5.1482e-07, -3.3258e-06,  6.7230e+00,  9.0000e+00,
           7.8574e+00,  6.2544e-07, -3.4934e-07,  7.6480e+00,  8.0000e+00,
           3.3232e-07, -1.6722e-06,  3.9642e-06,  2.1938e+00],
         [ 1.6252e+00, -1.9704e-05,  9.3346e-06,  6.0000e+00,  1.3789e+00,
           4.9415e+00, -7.2935e-06, -7.4822e-07,  6.0000e+00,  1.0375e+01,
          -1.0811e-05, -5.4580e-06,  1.2451e-05,  3.5627e+00],
         [ 3.0176e+00, -4.4808e-07,  1.1933e-08,  4.9478e+00,  2.4955e+00,
           4.9079e+00, -4.3189e-07,  1.4874e-06,  8.2671e+00,  9.0000e+00,
          -3.2392e-07, -9.0291e-07, -8.1849e-07,  3.6097e+00]],

        [[ 4.2925e+00,  2.3660e-06,  5.4283e-07,  6.6229e+00,  1.4000e+01,
           1.1795e+01,  2.1435e-08, -8.7306e-07,  7.7734e+00,  6.0000e+00,
           4.1899e-07,  2.7255e-07,  6.4913e-07,  1.8992e+00],
         [ 3.1877e+00,  7.9322e-06,  2.6586e-06,  1.5331e+00,  6.5331e+00,
           5.0000e+00, -1.1855e-06,  6.2542e-06,  3.1877e+00,  2.8661e-06,
           5.2284e-06,  2.4093e-06, -3.0814e-06,  5.4180e-06],
         [ 1.8542e-01, -1.4620e-06, -2.3625e-06,  2.5045e+00,  9.8016e+00,
           8.0000e+00,  5.2522e-08, -2.2418e-06,  3.5893e-01,  8.7635e-01,
          -2.2142e-06,  4.0179e-06, -1.1178e-06,  2.3244e-05],
         [-1.5973e-05,  8.7760e-06,  2.9659e-06,  3.2250e+00,  1.0336e+01,
           9.0000e+00,  7.5405e-07,  7.3980e-06,  4.8230e-01,  2.3717e+00,
           6.5460e-06, -2.8450e-06, -4.7256e-06,  3.7221e-05],
         [ 8.6219e+00, -3.8697e-07,  2.4101e-06,  6.8625e+00,  1.1750e+01,
           7.0000e+00, -8.0752e-07,  1.0097e-06,  1.2055e+01,  5.0000e+00,
          -1.8323e-06,  1.3786e-06, -1.0227e-06,  5.4603e-01],
         [ 2.6987e+00, -8.2139e-07,  2.7406e-06,  1.1439e+01,  1.0000e+01,
           7.3888e+00, -1.8454e-06,  3.3056e-06,  4.3085e+00,  9.0432e+00,
           1.1859e-06, -1.3578e-06,  2.9696e-08,  1.3944e+00],
         [ 3.1435e+00, -9.9087e-07, -2.4827e-06,  3.5852e+00,  9.0000e+00,
           1.1535e+01,  9.4719e-07, -2.1568e-06,  5.8619e+00,  5.4866e+00,
           8.2786e-07, -1.1095e-06,  6.6270e-06,  3.3523e+00],
         [ 3.0170e+00,  1.5823e-06,  1.5144e-06,  5.4644e+00,  7.0000e+00,
           6.6773e+00,  3.1825e-07,  5.7100e-07,  6.0163e+00,  6.7229e+00,
           2.8329e-06, -4.5524e-07, -9.5915e-07,  1.4181e+00],
         [ 3.1500e+00,  1.4650e-06, -1.8538e-06,  1.0304e+01,  1.0000e+01,
           5.9451e+00,  1.2376e-05,  1.8751e-06,  4.8881e+00,  7.0804e+00,
           2.9069e-07,  2.9661e-06, -2.2033e-06,  9.0643e-01],
         [ 3.0448e+00, -3.7740e-06, -3.8693e-06,  4.5954e+00,  8.0000e+00,
           9.3439e+00,  1.2930e-06, -1.4096e-06,  7.1902e+00,  7.4330e+00,
          -3.4331e-06,  4.3735e-07,  1.7277e-06,  2.6517e+00],
         [ 3.0505e+00,  1.2564e-07, -2.6926e-06,  6.8114e+00,  8.0000e+00,
           5.3599e+00, -4.1216e-07, -1.8258e-06,  7.5237e+00,  7.0000e+00,
           9.3171e-07, -3.6995e-06,  4.7579e-06,  1.6446e+00],
         [ 3.0575e+00, -1.9230e-06,  1.8548e-06,  3.2330e+00,  6.0000e+00,
           9.1298e+00, -6.6510e-07,  1.5640e-06,  6.5702e+00,  6.2834e+00,
           2.9070e-06,  1.5231e-06,  3.5946e-06,  3.5922e+00],
         [ 3.3881e+00,  1.2907e-05, -3.1890e-06,  1.0535e+01,  1.1000e+01,
           6.9284e+00, -2.0805e-06, -1.2162e-05,  6.2635e+00,  8.4703e+00,
           4.7037e-06, -5.9761e-06, -1.4201e-05,  8.6891e-01],
         [ 2.9144e+00,  4.4322e-07,  6.9885e-06,  1.9822e+00,  7.7101e+00,
           6.0000e+00,  3.4030e-06,  1.3620e-05,  3.1519e+00,  5.0943e-01,
          -5.0432e-06,  9.5567e-06, -1.9805e-05,  7.1921e-05],
         [ 7.4369e+00, -6.0397e-06,  1.0812e-05,  8.2169e-01,  8.0000e+00,
           1.0258e+01, -7.8137e-06,  8.1556e-06,  7.6289e+00,  4.6075e-01,
          -7.9328e-06,  2.3485e-05,  7.2108e-06,  2.8108e+00],
         [ 5.9083e+00, -5.6237e-07,  2.7807e-06,  6.0530e+00,  1.1000e+01,
           7.2653e+00, -2.2615e-06,  2.6164e-06,  1.1678e+01,  7.0000e+00,
          -2.6553e-06,  2.9019e-06, -3.1055e-07,  1.0876e+00],
         [ 5.9106e+00,  3.3319e-06, -2.1253e-06,  6.6912e+00,  1.0000e+01,
           7.0000e+00, -1.4637e-07, -1.0125e-06,  8.4268e+00,  5.0671e+00,
           4.8526e-06, -2.8947e-06,  2.9473e-06,  1.1403e+00],
         [ 3.0546e+00, -7.9457e-06,  1.8367e-06,  8.7429e+00,  9.0000e+00,
           5.3034e+00,  1.8033e-06,  2.4502e-06,  5.7562e+00,  6.7294e+00,
          -2.6146e-06,  6.1793e-06, -2.8228e-06,  1.0186e+00],
         [ 3.0629e+00, -4.1003e-06,  6.9539e-08,  6.0709e+00,  9.0000e+00,
           7.7747e+00,  2.1133e-06,  1.4237e-06,  6.9366e+00,  7.0474e+00,
          -5.6906e-07,  4.2393e-07, -2.4974e-06,  1.6718e+00],
         [ 3.2341e+00, -4.5804e-07,  6.3090e-07,  7.4243e+00,  3.0000e+00,
           5.3713e+00,  2.8727e-06, -3.6900e-07,  6.3479e+00,  9.2185e+00,
          -4.2097e-07, -3.5168e-07, -4.1208e-06,  3.6909e+00],
         [ 3.0063e+00,  1.6219e-06,  7.5646e-07,  5.1145e+00,  9.0000e+00,
           6.7658e+00, -3.3705e-06, -1.3070e-07,  7.6100e+00,  6.0000e+00,
           3.4429e-06, -3.0599e-06, -3.5696e-07,  1.4840e+00],
         [ 5.2946e+00,  1.4426e-06, -3.0679e-07,  4.0490e+00,  8.0000e+00,
           7.9094e+00,  4.3738e-07, -8.0532e-07,  1.2539e+01,  9.0000e+00,
           4.5401e-07,  6.1904e-07,  3.5174e-07,  2.2029e+00],
         [ 4.8468e+00,  5.1223e-07,  6.2954e-07,  3.3249e-01,  7.0000e+00,
           1.0915e+01,  9.0048e-07,  1.5132e-07,  4.8468e+00, -8.1725e-08,
           2.9858e-06,  1.7728e-06,  3.3230e-06,  4.2477e+00],
         [ 3.2541e+00, -2.7138e-06,  2.0008e-06,  7.6381e+00,  8.0000e+00,
           6.6221e+00,  1.9206e-07, -6.6519e-07,  5.0182e+00,  6.7579e+00,
           3.3358e-06,  2.4150e-06,  4.6770e-06,  1.2664e+00],
         [ 4.3736e+00,  6.2593e-07, -3.3661e-06,  7.3241e+00,  1.2000e+01,
           8.8260e+00, -1.4572e-06, -3.1222e-06,  7.9696e+00,  6.7661e+00,
           3.7188e-06, -3.4281e-06,  2.4144e-06,  9.8013e-01],
         [ 4.2328e+00,  1.0282e-06, -1.5595e-06,  6.0033e+00,  9.0000e+00,
           7.0546e+00, -7.0455e-07, -1.4975e-06,  8.2870e+00,  6.0000e+00,
           1.1982e-06, -4.7750e-07,  1.7113e-06,  2.1121e+00],
         [ 1.1867e+00, -2.3145e-06, -4.7033e-06,  1.0000e+01,  9.8792e+00,
           8.0000e+00, -3.6254e-06,  1.7619e-07,  2.3928e+00,  8.5030e+00,
           1.2782e-06, -6.5029e-06, -3.1835e-07,  8.2393e-01],
         [ 4.8170e+00, -8.7245e-08,  2.0495e-08,  2.9051e+00,  7.0000e+00,
           9.7956e+00, -9.3116e-08, -8.6266e-08,  7.9994e+00,  5.6886e+00,
          -1.4757e-07, -2.5926e-07, -3.2040e-07,  3.1944e+00],
         [ 3.1078e+00,  1.2681e-06,  7.9024e-07,  8.8352e+00,  1.1000e+01,
           8.4643e+00,  1.2687e-06,  1.5218e-08,  5.8031e+00,  7.3662e+00,
          -1.6731e-07,  1.8084e-06, -1.9228e-07,  1.6286e+00],
         [ 3.0347e+00,  5.1482e-07, -3.3258e-06,  6.7230e+00,  9.0000e+00,
           7.8574e+00,  6.2544e-07, -3.4934e-07,  7.6480e+00,  8.0000e+00,
           3.3232e-07, -1.6722e-06,  3.9642e-06,  2.1938e+00],
         [ 2.0639e-06,  1.1035e-05, -1.0654e-05,  6.0000e+00,  7.0000e+00,
           9.9334e+00, -4.3502e-07,  2.0664e-06,  7.4532e-06,  6.0001e+00,
          -8.9760e-06, -1.7700e-05, -7.2864e-06,  2.9334e+00],
         [ 2.2910e+00, -1.6778e-06, -6.4042e-07,  2.5636e-01,  1.1000e+01,
           1.5248e+01,  1.7301e-06, -8.9817e-07,  2.2910e+00, -3.2517e-06,
           2.4403e-06,  1.0425e-06,  1.7847e-06,  4.5042e+00]]])


In [9]:
leftovers = torch.tensor([[2.1854e+00, 0.0000e+00, 0.0000e+00, 3.2583e+00, 1.0209e+01, 6.3661e+00,
         0.0000e+00],
        [8.7306e-07, 3.2266e+00, 1.4305e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.1101e+01],
        [1.2789e+01, 3.6610e-07, 2.5236e-06, 6.3088e+00, 5.0878e+00, 2.4307e+00,
         1.3029e-06],
        [0.0000e+00, 5.8123e+00, 7.0000e+00, 0.0000e+00, 0.0000e+00, 3.0814e-06,
         1.4000e+01],
        [5.6873e+00, 1.4067e-06, 1.3643e-07, 6.8086e+00, 1.3644e+01, 7.8888e+00,
         1.6188e-06],
        [2.2418e-06, 7.6411e+00, 8.1236e+00, 2.2142e-06, 0.0000e+00, 1.1178e-06,
         1.3000e+01],
        [4.1197e+00, 0.0000e+00, 7.9659e-07, 1.0299e+00, 9.9570e+00, 9.0000e+00,
         4.6463e-07],
        [0.0000e+00, 9.5177e+00, 7.6283e+00, 0.0000e+00, 2.8450e-06, 4.7256e-06,
         1.0000e+01],
        [5.2574e+00, 2.6261e-06, 1.7734e-06, 7.4179e+00, 5.8086e+00, 9.7669e-01,
         3.4839e-06],
        [0.0000e+00, 4.9449e+00, 2.8610e-06, 1.8323e-06, 0.0000e+00, 1.0227e-06,
         7.4540e+00],
        [7.3900e+00, 9.3522e-07, 2.5264e-06, 4.2035e+00, 6.4807e+00, 2.5635e+00,
         7.4609e-07],
        [0.0000e+00, 2.6915e+00, 2.9568e+00, 0.0000e+00, 1.3578e-06, 0.0000e+00,
         9.6056e+00],
        [6.7576e+00, 1.2774e-07, 1.7929e-07, 1.4888e+00, 5.8392e+00, 7.8533e+00,
         6.8560e-07],
        [2.1568e-06, 2.1381e+00, 2.5134e+00, 0.0000e+00, 1.1095e-06, 0.0000e+00,
         1.0648e+01],
        [1.1641e+01, 0.0000e+00, 1.8393e-06, 8.3600e-01, 5.2231e+00, 6.2562e+00,
         8.2089e-08],
        [0.0000e+00, 2.9837e+00, 3.2771e+00, 0.0000e+00, 4.5524e-07, 9.5915e-07,
         9.5819e+00],
        [9.0000e+00, 4.7865e-06, 0.0000e+00, 1.3447e+01, 1.0000e+01, 2.3563e+00,
         1.9358e-06],
        [0.0000e+00, 2.1119e+00, 1.9196e+00, 0.0000e+00, 0.0000e+00, 2.2033e-06,
         1.3094e+01],
        [4.6072e+00, 1.7548e-06, 4.4187e-07, 4.0065e+00, 1.4305e-06, 3.0948e+00,
         0.0000e+00],
        [1.4096e-06, 2.8098e+00, 2.5670e+00, 3.4331e-06, 0.0000e+00, 0.0000e+00,
         7.3483e+00],
        [9.9184e+00, 4.5938e-06, 2.1571e-06, 7.7322e+00, 4.7497e+00, 2.8563e+00,
         0.0000e+00],
        [1.8258e-06, 4.7630e-01, 0.0000e+00, 0.0000e+00, 3.6995e-06, 0.0000e+00,
         1.3355e+01],
        [5.8798e+00, 4.9925e-07, 0.0000e+00, 4.0553e+00, 0.0000e+00, 6.8154e+00,
         0.0000e+00],
        [0.0000e+00, 2.4298e+00, 2.7166e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         8.4078e+00],
        [3.9609e+00, 3.8409e-06, 0.0000e+00, 6.7566e+00, 7.4333e+00, 2.8329e+00,
         0.0000e+00],
        [1.2162e-05, 2.7365e+00, 3.5297e+00, 0.0000e+00, 5.9761e-06, 1.4201e-05,
         8.1311e+00],
        [8.1190e+00, 2.8993e-07, 6.7593e-07, 8.0000e+00, 9.8106e+00, 4.7684e-07,
         3.2332e-07],
        [0.0000e+00, 6.8481e+00, 6.4906e+00, 5.0432e-06, 0.0000e+00, 1.9805e-05,
         1.3000e+01],
        [2.6971e+00, 3.8352e-06, 3.7477e-07, 5.6036e+00, 6.8630e+00, 9.3211e+00,
         0.0000e+00],
        [0.0000e+00, 8.3711e+00, 3.5393e+00, 7.9328e-06, 0.0000e+00, 0.0000e+00,
         7.1892e+00],
        [4.5773e+00, 0.0000e+00, 4.9312e-06, 8.5948e+00, 8.0063e+00, 3.0104e+00,
         5.3627e-06],
        [0.0000e+00, 1.3225e+00, 8.1062e-06, 2.6553e-06, 0.0000e+00, 3.1055e-07,
         8.9124e+00],
        [7.3738e+00, 6.8111e-06, 0.0000e+00, 1.1678e+01, 1.0000e+01, 2.0509e+00,
         4.4327e-06],
        [1.0125e-06, 2.5732e+00, 2.9329e+00, 0.0000e+00, 2.8947e-06, 0.0000e+00,
         9.8597e+00],
        [1.2761e+01, 0.0000e+00, 0.0000e+00, 5.7990e-01, 4.5098e+00, 3.5845e+00,
         7.2834e-07],
        [0.0000e+00, 2.2438e+00, 2.2706e+00, 2.6146e-06, 0.0000e+00, 2.8228e-06,
         1.1981e+01],
        [6.0188e+00, 0.0000e+00, 0.0000e+00, 5.6990e+00, 6.0373e+00, 5.8969e+00,
         0.0000e+00],
        [0.0000e+00, 2.0634e+00, 1.9526e+00, 5.6906e-07, 0.0000e+00, 2.4974e-06,
         1.0328e+01],
        [7.6937e+00, 0.0000e+00, 1.7592e-06, 5.0461e+00, 1.1921e-06, 7.4409e+00,
         2.4780e-06],
        [3.6900e-07, 4.6521e+00, 3.7815e+00, 4.2097e-07, 3.5168e-07, 4.1208e-06,
         2.3091e+00],
        [8.1228e+00, 0.0000e+00, 0.0000e+00, 7.5529e+00, 5.2982e+00, 4.2586e+00,
         2.7966e-07],
        [1.3070e-07, 3.9002e-01, 0.0000e+00, 0.0000e+00, 3.0599e-06, 3.5696e-07,
         1.4516e+01],
        [1.7054e+00, 0.0000e+00, 3.0679e-07, 1.0951e+01, 0.0000e+00, 2.0906e+00,
         0.0000e+00],
        [8.0532e-07, 2.4609e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         3.7971e+00],
        [2.9845e+00, 0.0000e+00, 0.0000e+00, 9.6645e+00, 5.4726e+00, 7.6785e+00,
         1.6691e-06],
        [0.0000e+00, 7.1532e+00, 8.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         5.7523e+00],
        [7.8729e+00, 0.0000e+00, 1.3066e-06, 3.1546e+00, 5.3325e+00, 5.6365e+00,
         0.0000e+00],
        [6.6519e-07, 2.9818e+00, 3.2421e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0734e+01],
        [6.7564e+00, 1.2321e-05, 4.4858e-06, 3.6427e+00, 8.5710e+00, 4.8666e+00,
         0.0000e+00],
        [3.1222e-06, 5.0304e+00, 5.2339e+00, 0.0000e+00, 3.4281e-06, 0.0000e+00,
         4.0199e+00],
        [7.4885e+00, 0.0000e+00, 1.0362e-06, 7.3238e+00, 6.1653e+00, 4.4915e+00,
         1.0114e-06],
        [1.4975e-06, 2.7130e+00, 0.0000e+00, 0.0000e+00, 4.7750e-07, 0.0000e+00,
         1.0888e+01],
        [9.3577e+00, 1.5928e-05, 0.0000e+00, 2.7865e+00, 3.3841e+00, 3.7148e-01,
         0.0000e+00],
        [0.0000e+00, 3.6072e+00, 3.4970e+00, 0.0000e+00, 6.5029e-06, 3.1835e-07,
         1.1176e+01],
        [5.6543e+00, 8.0668e-07, 5.8021e-07, 4.5370e+00, 1.9073e-06, 4.2471e+00,
         0.0000e+00],
        [8.6266e-08, 3.0006e+00, 4.3114e+00, 1.4757e-07, 2.5926e-07, 3.2040e-07,
         5.8056e+00],
        [6.7292e+00, 4.6815e-06, 1.0527e-06, 2.5555e+00, 6.2199e+00, 3.4775e+00,
         0.0000e+00],
        [0.0000e+00, 3.1969e+00, 3.6338e+00, 1.6731e-07, 0.0000e+00, 1.9228e-07,
         8.3714e+00],
        [2.9653e+00, 0.0000e+00, 3.3258e-06, 7.2770e+00, 1.9073e-06, 3.1426e+00,
         0.0000e+00],
        [3.4934e-07, 1.3520e+00, 0.0000e+00, 0.0000e+00, 1.6722e-06, 0.0000e+00,
         1.0806e+01],
        [1.2375e+01, 1.9704e-05, 0.0000e+00, 2.0981e-05, 5.6211e+00, 8.0585e+00,
         7.2935e-06],
        [0.0000e+00, 6.0000e+00, 9.9999e+00, 8.9760e-06, 1.7700e-05, 7.2864e-06,
         5.0666e+00],
        [9.8235e-01, 4.4808e-07, 0.0000e+00, 4.0522e+00, 8.5045e+00, 1.1092e+01,
         4.3189e-07],
        [8.9817e-07, 1.0709e+01, 9.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         3.4958e+00]])

In [13]:
print(f'true_bid shape: {true_bids.shape}')
print(f'central_alloc shape: {central_alloc.shape}')
print(f'leftovers shape: {leftovers.shape}')

true_bid shape: torch.Size([32, 2, 7])
central_alloc shape: torch.Size([2, 32, 14])
leftovers shape: torch.Size([64, 7])


In [20]:
N_HOS = 2
N_TYP = 7
internal_s = torch.tensor(np.load('type_matrix/ashlagi_7_type.npy'),
                          requires_grad=False, dtype=torch.float32)

central_s = torch.tensor(mn.convert_internal_S(internal_s.numpy(), N_HOS),
                         requires_grad=False, dtype=torch.float32)

int_structures = internal_s.shape[1]

In [23]:
x_int = cp.Variable( int_structures )
int_w = cp.Parameter( int_structures )
int_b = cp.Parameter( N_TYP )

int_constraints = [x_int >= 0, internal_s.numpy() @ x_int <= int_b]  # constraint for positive allocation and less than true bid
objective = cp.Maximize( (int_w.T @ x_int) )
problem = cp.Problem(objective, int_constraints)
internal_layer =  CvxpyLayer(problem, parameters=[int_w, int_b], variables=[x_int])

In [35]:

W = torch.ones(64, int_structures)
B = leftovers.view(64, N_TYP)
internal_layer(W, B, solver_args={'max_iters': 50000, 'verbose': False, 'eps': 1e-3, 'scale': 5.0})

(tensor([[ 1.5264e-11,  4.9082e-11,  7.8845e-11,  3.2583e+00,  6.3661e+00,
           7.2928e-11],
         [-7.8634e-07,  7.6679e-07,  8.7469e-07,  5.8777e-07, -2.9415e-07,
           2.6074e-06],
         [ 2.4240e-07,  5.1161e-07,  9.6247e-07,  3.4437e+00,  1.6441e+00,
           7.0775e-07],
         [ 5.4564e-06,  5.8123e+00,  4.7380e-06, -1.0495e-06,  4.3386e-06,
          -2.1217e-06],
         [ 1.6593e-07,  2.8901e-07,  2.2935e-07,  6.5485e+00,  7.0955e+00,
           5.4873e-07],
         [-6.2540e-06,  7.6411e+00, -2.9982e-06,  1.2102e-06, -9.2062e-06,
           1.0457e-05],
         [-1.7907e-07,  2.1998e-07,  7.4167e-07,  9.9347e-01,  8.9635e+00,
           6.0285e-07],
         [-4.1453e-07,  7.6283e+00, -2.1658e-08,  6.6636e-07,  1.4944e-06,
           3.4710e-06],
         [-8.0332e-08,  2.0634e-06, -4.8303e-07,  5.1998e+00,  6.0881e-01,
           3.6892e-06],
         [ 2.8955e-06, -2.2233e-06,  3.9576e-06,  1.5540e-06, -1.4495e-06,
           1.1350e-07],
         [

In [279]:
central_s[:, 7]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.])

In [259]:
central_s[:, 11].numpy().reshape(2, 7).nonzero()


(array([1, 1], dtype=int64), array([2, 3], dtype=int64))

In [239]:
calc_n_edges(batch[0, :], ashlagi_compat_dict)

42.0

In [238]:

hos1_probs = [0.25, 0, 0, 0.25, 0.25, 0.25, 0]
hos2_probs = [0, 0.33, 0.33, 0, 0, 0, 0.34]
hos_gen_lst = [GenericTypeHospital(hos1_probs, 10),
               GenericTypeHospital(hos2_probs, 10)]

generator = hg.ReportGenerator(hos_gen_lst, (N_HOS, N_TYP))
internal_s = torch.tensor(np.load('ashlagi_7_type.npy'),
                                  requires_grad=False, dtype=torch.float32)
int_structures = internal_s.shape[1]
central_s = torch.tensor(mn.convert_internal_S(internal_s.numpy(), N_HOS),
                         requires_grad=False, dtype=torch.float32)
batch = next(generator.generate_report(3))
ashlagi_compat_dict = {}
for i in range(1, N_TYP - 1):
    ashlagi_compat_dict[i] = []
    ashlagi_compat_dict[i].append(i - 1)
    ashlagi_compat_dict[i].append(i + 1)
ashlagi_compat_dict[0] = [1]
ashlagi_compat_dict[N_TYP - 1] = [N_TYP - 2]

In [222]:
def calc_n_edges(bids, compat_dict):
    """
    bids: [n_hos, n_types]
    """
    n_hos = bids.shape[0]
    n_types = bids.shape[1]
    type_totals = bids.sum(axis=0)
    n_edges = 0
    for t in range(n_types):
        num_patients = type_totals[t]
        compat_types = compat_dict[t]
        # compatible types
        for compat_t in compat_types:
            if t == compat_t:
                n_edges += type_totals[t] * (type_totals[t] - 1)
            else:
                n_edges += type_totals[t] * type_totals[compat_t]
    return n_edges / 2.0


In [ ]:
def create_weights(bids, compat_dict):
    

In [253]:
print(internal_hard_s.shape)
internal_hard_s

torch.Size([7, 6])


tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.]])

In [215]:
class GenericTypeHospital:
    """
    Given a probability distribution for types generate n patients according to that distribution
    """
    def __init__(self, type_probs, n_patients):
        self.n_types = len(type_probs)
        self.probs = type_probs
        self.k = n_patients
    
    def generate(self, batch_size):
        truthful_bid = np.zeros((batch_size, self.n_types))
        for batch in range(batch_size):
            for i in range(self.k):
                rand_type = np.random.choice(np.arange(self.n_types), 1, p=self.probs)
                truthful_bid[batch, int(rand_type)] += 1.0
                
        return truthful_bid
        
        
    

In [66]:
np.save('ashlagi_7_type.npy', internal_hard_s.numpy())

In [251]:
n_types = 7
internal_hard_s = torch.zeros(n_types, n_types - 1)
for i in range(n_types - 1):
    internal_hard_s[i, i] = 1
    internal_hard_s[i + 1, i]= 1


In [280]:
torch.load('two_two_test2020-02-04_19-38-10/')[0].abs().mean()

tensor(0.3562)

In [286]:
args

{'main_lr': 0.1,
 'main_iter': 5,
 'init_iter': 100,
 'batchsize': 4,
 'nbatch': 2,
 'misreport_iter': 10,
 'misreport_lr': 1.0,
 'random_seed': 0,
 'control_strength': 5.0}

In [285]:
import json
with open('two_two_test2020-02-04_19-38-10/argfile.json') as args_file:
    args = json.load(args_file)